<a href="https://colab.research.google.com/github/mtzig/NLP_CTF/blob/main/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Baseline Toxicity Classifier
Thomas Zeng
9/27/22

## Colab setup

This section is only pertinent if the notebook is run in Colab and not on a local machine.

If you're using colab, make sure to run below code to clone the repo

In [2]:
!git clone https://github.com/mtzig/NLP_CTF.git
%cd /content/NLP_CTF/

Cloning into 'NLP_CTF'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 162 (delta 19), reused 34 (delta 16), pack-reused 123
Receiving objects: 100% (162/162), 102.56 MiB | 22.03 MiB/s, done.
Resolving deltas: 100% (83/83), done.
Updating files: 100% (29/29), done.
[Errno 2] No such file or directory: '/content/NLP_CTF/'
/Users/teaganjohnson/Desktop/NLP_CTF


Download Word2Vec Embeddings

In [11]:
%cd ./data
# !wget -O 'GoogleNews-vectors-negative300.bin.gz' 'https://drive.google.com/u/0/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM&export=download&confirm=t&uuid=e1f49911-ab4d-44ba-af00-f6733ccabb98'
# !gzip -d 'GoogleNews-vectors-negative300.bin.gz'
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1JXm1N6SHmzIawgH7Aa4Ag-ZVuqLX7ba7' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1JXm1N6SHmzIawgH7Aa4Ag-ZVuqLX7ba7" -O GoogleNews-vectors-negative300.bin && rm -rf /tmp/cookies.txt
%cd ..

/Users/teaganjohnson/Desktop/NLP_CTF/data
--2022-10-03 13:34:43--  https://docs.google.com/uc?export=download&confirm=t&id=1JXm1N6SHmzIawgH7Aa4Ag-ZVuqLX7ba7
Resolving docs.google.com (docs.google.com)... 142.250.190.14
Connecting to docs.google.com (docs.google.com)|142.250.190.14|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8nsrvua6o5em6vg7nkfu6fcndcg453g0/1664822025000/15857340408018396550/*/1JXm1N6SHmzIawgH7Aa4Ag-ZVuqLX7ba7?e=download&uuid=4e8c9954-c006-4d8a-b5ea-36a2026a262c [following]
--2022-10-03 13:34:43--  https://doc-08-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8nsrvua6o5em6vg7nkfu6fcndcg453g0/1664822025000/15857340408018396550/*/1JXm1N6SHmzIawgH7Aa4Ag-ZVuqLX7ba7?e=download&uuid=4e8c9954-c006-4d8a-b5ea-36a2026a262c
Resolving doc-08-7o-docs.googleusercontent.com (doc-08-7o-docs.googleusercontent.com)... 142.250.191

Colab does not have the Python library `transformers` (which I use in below code) automatically installed, so we meed to manually install when we start up instance.

In [4]:
!pip install transformers
!pip install --upgrade gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 20.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.0 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 18.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 1.9 MB/s eta 0:00:00


## Notebook Setup

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import torch
from process_data import get_jigsaw_datasets, init_embed_lookup
from models import CNNClassifier
from train_eval import train, evaluate
from torch.utils.data import DataLoader

## Data Initialization

Pytorch requires its datasets to be ascessible following the [datasets api](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files).

Below I wrote a simple function to load in the [Jigsaw Dataset](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge) that the paper [Counterfactual Fairness in
Text Classification through Robustness](https://dl.acm.org/doi/pdf/10.1145/3306618.3317950) used to train its toxicity classifier.

I use only a very small subset of the available data here for demonstration purposes. Specificaly 256 comments (128 toxic and 128 nontoxic) sampled randomly for the train set and test set respectively.

In [27]:
train_data, test_data = get_jigsaw_datasets(device=DEVICE, data_set='blind') #demo_mode=True only loads a subset of entire dataset so as to make training faster for demonstration purposes

PyTorch models receive data for training and inference through a dataloader. A dataloader samples from a dataset and returns a batch of samples each time it is called.

In [28]:
train_loader =  torch.utils.data.DataLoader(train_data, batch_size=64)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [31]:
len(train_loader.dataset)

159571

## Model and Training Stuff Initialization

In [29]:
pretrained_embed = torch.from_numpy(init_embed_lookup().vectors)


In [32]:

model = CNNClassifier(pretrained_embed,device=DEVICE)

An epoch is the number of times you go through your datase during training. That is you have trained for 1 epoch when you have seen every sample in your training dataset once.<br>
The loss function is the training objective we want our model to minimize.<br>
The optimizer is used at every time step i.e. everyime we compute the loss and its gradient. It is used to update the model weights.

In [33]:
epochs = 5
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters())

## Train and Evaluation

For traing, we train for 10 epochs. <br>
In general, you should (or more specifically are required to) train and evaluate using different datasets.

In [36]:
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}')
    train(train_loader, model, loss_fn, optimizer, verbose=True)

Epoch 1/5


100%|██████████| 2494/2494 [04:55<00:00,  8.43it/s]


Average training loss: 0.1266589334959787
Epoch 2/5


  4%|▍         | 103/2494 [00:12<04:42,  8.46it/s]


KeyboardInterrupt: 

We first evaluate our results on our train data

In [37]:
_ = evaluate(train_loader, model, get_loss=True, verbose=True)

100%|██████████| 2494/2494 [01:27<00:00, 28.51it/s]

Loss: 0.09106984734535217
Accuracy: 0.9655764518615538, Sensitivity: 0.7589904537727213, Specificity: 0.9874754811924285


We then evaluate our reults on the test data:

In [38]:
_ = evaluate(test_loader, model, get_loss=True, verbose=True)

100%|██████████| 1000/1000 [00:35<00:00, 27.80it/s]

Loss: 0.18660908937454224
Accuracy: 0.9213948544812279, Sensitivity: 0.813136288998358, Specificity: 0.9327839966832504
